In [1]:
from math import sqrt
import fiona  # pour lire les shapefile
import pyproj

In [5]:
import numpy as np
import pandas as pd

## Recherche des données de population des pays frontaliers

### Import des données de populations de la plupart des pays d'Europe

In [6]:
geo_grid = pd.read_csv('data/geostat/GEOSTAT_grid_POP_1K_2011_V2_0_1.csv')

/home/savinien/anaconda3/envs/py35_1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
geo_grid["CNTR_CODE"].unique()

array(['DE', 'ES', 'CH', 'FR', 'IE', 'FI', 'MT', 'CZ', 'PL', 'LI', 'LT',
       'UK', 'SE', 'BG', 'XK*', 'EL', 'AT', 'IT', 'BE', 'PT', 'SI', 'NO',
       'LV', 'HR', 'SK', 'HU', 'NL', 'EE', 'RO', 'DK', 'AL'], dtype=object)

### Import des données des des pays restants ( seul le Luxembourg nous interesse ici)

In [8]:
geo_grid2 = pd.read_csv('data/geostat/JRC-GHSL_AIT-grid-POP_1K_2011.csv')

In [9]:
geo_grid2.CNTR_CODE.unique()

array(['CY', 'LU', 'IM', 'MC', 'SM', 'AD', 'VA', 'BA', 'IS', 'ME', 'MK',
       'RS'], dtype=object)

In [10]:
geo_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026602 entries, 0 to 2026601
Data columns (total 7 columns):
TOT_P           int64
GRD_ID          object
CNTR_CODE       object
METHD_CL        object
YEAR            int64
DATA_SRC        object
TOT_P_CON_DT    object
dtypes: int64(2), object(5)
memory usage: 108.2+ MB


#### Sélection du Luxembourg uniquement

In [11]:
subframeLUX = geo_grid2[geo_grid2.CNTR_CODE == 'LU']

#### Concatenation avec le dataframe des principaux pays

In [12]:
geo_grid_merged = pd.concat([geo_grid, subframeLUX])

In [13]:
geo_grid_merged.shape

(2028301, 7)

#### Decompression puis Import de données shapefile

In [31]:
import tarfile

In [32]:
with tarfile.open("data/geostat/GEOSTATReferenceGrid/Grid_ETRS89_LAEA_1K-ref_GEOSTAT_POP_2011_V2_0_1.shp.tar.gz") as tar:
    tar.extractall(path="data/geostat/GEOSTATReferenceGrid/")

In [33]:
with tarfile.open("data/geostat/GEOSTATReferenceGrid/Grid_ETRS89_LAEA_1K-ref_GEOSTAT_POP_2011_V2_0_1.dbf.tar.gz") as tar:
    tar.extractall(path="data/geostat/GEOSTATReferenceGrid/")

In [34]:
x_laea = []
y_laea = []
grid_shape = []

with fiona.open('data/geostat/GEOSTATReferenceGrid/Grid_ETRS89_LAEA_1K-ref_GEOSTAT_POP_2011_V2_0_1.shp') as shapeFile:
    for i, point in enumerate(shapeFile):
        x_laea.append(point['geometry']['coordinates'][0][0][0])
        y_laea.append(point['geometry']['coordinates'][0][0][1])
        grid_shape.append(str(point['properties']['GRD_ID']))

#### Création d'un dataframe à partir des x, y et id de grid des shapefiles

In [35]:
df_shapefiles = pd.DataFrame({'x':x_laea, 'y':y_laea, 'grid':grid_shape})

In [37]:
df_shapefiles.head()

,grid,x,y
0,1kmN1760E2636,2636000.0,1760000.0
1,1kmN1954E2636,2636000.0,1954000.0
2,1kmN1961E2636,2636000.0,1961000.0
3,1kmN1950E2637,2637000.0,1950000.0
4,1kmN1951E2637,2637000.0,1951000.0


In [38]:
lambert=pyproj.Proj("+init=EPSG:3035")

#### Création d'un dataframe pour les centrales (avec latitudes et longitudes)

In [39]:
longitudes=[2.875,-0.69083,5.27083,6.21806,0.17028,4.79056,0.6528,4.75667,2.51667,7.563036,-1.88167,0.84528,2.135,3.51778,0.63528,1.21194,4.75528,1.58349,4.72249]
latitudes=[47.50972,45.25611,45.79833,49.41583,47.2306,50.09,46.45667,44.63306,47.73306,47.903108,49.53639,44.10667,51.01444,48.51528,49.85778,49.97611,45.40444,47.723982,44.335698]

In [49]:
df_centrales = (pd.DataFrame({'longitudes':longitudes, 'latitudes':latitudes}))

In [50]:
df_centrales.head()

,latitudes,longitudes
0,47.50972,2.87500
1,45.25611,-0.69083
2,45.79833,5.27083
3,49.41583,6.21806
4,47.23060,0.17028


#### Conversion des latitudes et longitudes des centrales en x et y

In [51]:
df_centrales[['x', 'y']] = df_centrales[['longitudes', 'latitudes']].apply(lambda row: list(lambert(row[0],row[1])), axis=1)

In [52]:
df_centrales.head()

,latitudes,longitudes,x,y
0,47.50972,2.87500,3.784859e+06,2.736510e+06
1,45.25611,-0.69083,3.483758e+06,2.520596e+06
2,45.79833,5.27083,3.953121e+06,2.532309e+06
3,49.41583,6.21806,4.046662e+06,2.929604e+06
4,47.23060,0.17028,3.578537e+06,2.729031e+06


In [53]:
df_shapefiles.head()

,grid,x,y
0,1kmN1760E2636,2636000.0,1760000.0
1,1kmN1954E2636,2636000.0,1954000.0
2,1kmN1961E2636,2636000.0,1961000.0
3,1kmN1950E2637,2637000.0,1950000.0
4,1kmN1951E2637,2637000.0,1951000.0


#### Fusion (merge) des dataframes shapefile et geogrid, sur la clé 'GRD_ID' ('grid')

In [47]:
df_merged = (pd.merge(df_shapefiles, geo_grid_merged[['TOT_P','GRD_ID','CNTR_CODE']], how='left', left_on='grid', right_on='GRD_ID')
 .drop('GRD_ID', axis=1))

In [48]:
df_merged.head()

,grid,x,y,TOT_P,CNTR_CODE
0,1kmN1760E2636,2636000.0,1760000.0,12.0,PT
1,1kmN1954E2636,2636000.0,1954000.0,1.0,PT
2,1kmN1961E2636,2636000.0,1961000.0,6.0,PT
3,1kmN1950E2637,2637000.0,1950000.0,3.0,PT
4,1kmN1951E2637,2637000.0,1951000.0,110.0,PT


** Fonction d'ajout d'un champ de calcul de population dans un dataframe
à partir des colonnes x, y et d'une distance donnée**

In [54]:
def return_pop(row, km):
    mask = ((df_merged.x+500-row.x)**2
            +(df_merged.y+500-row.y)**2).apply(sqrt)< (1000*km)
    return df_merged[mask].TOT_P.sum()

In [55]:
df_centrales['popCentrale_10'] = df_centrales.apply(return_pop, args=(10,), axis=1)

In [56]:
df_centrales['popCentrale_20'] = df_centrales.apply(return_pop, args=(20,), axis=1)
df_centrales['popCentrale_30'] = df_centrales.apply(return_pop, args=(30,), axis=1)
df_centrales['popCentrale_100'] = df_centrales.apply(return_pop, args=(100,), axis=1)

In [57]:
df_centrales

,latitudes,longitudes,x,y,popCentrale_10,popCentrale_20,popCentrale_30,popCentrale_100
0,47.509720,2.875000,3.784859e+06,2.736510e+06,12736.0,50815.0,97251.0,1738957.0
1,45.256110,-0.690830,3.483758e+06,2.520596e+06,15581.0,67348.0,148757.0,2276468.0
2,45.798330,5.270830,3.953121e+06,2.532309e+06,42646.0,189795.0,705931.0,6137194.0
3,49.415830,6.218060,4.046662e+06,2.929604e+06,92989.0,388704.0,813932.0,4426623.0
4,47.230600,0.170280,3.578537e+06,2.729031e+06,29505.0,86334.0,171913.0,2719971.0
5,50.090000,4.790560,3.948525e+06,3.010752e+06,22972.0,70071.0,182047.0,6590937.0
6,46.456670,0.652800,3.604317e+06,2.638887e+06,11950.0,50061.0,206764.0,1798093.0
7,44.633060,4.756670,3.904536e+06,2.405940e+06,52694.0,139678.0,261372.0,3195321.0
8,47.733060,2.516670,3.760433e+06,2.763887e+06,17727.0,61923.0,125007.0,2538767.0
9,47.903108,7.563036,4.138736e+06,2.757394e+06,50346.0,241690.0,962487.0,7442318.0


In [58]:
nomsCentrales=['Belleville','Blayais','Bugey','Cattenom','Chinon-B','Chooz-B','Civaux','Cruas','Dampierre','Fessenheim','Flamanville','Golfech','Gravelines','Nogent','Paluel','Penly','Saint-Alban','St. Laurent','Tricastin']

In [59]:
df_centrales['nom'] = nomsCentrales

In [60]:
df_centrales.sort_values('popCentrale_20', ascending=False)

,latitudes,longitudes,x,y,popCentrale_10,popCentrale_20,popCentrale_30,popCentrale_100,nom
3,49.415830,6.218060,4.046662e+06,2.929604e+06,92989.0,388704.0,813932.0,4426623.0,Cattenom
16,45.404440,4.755280,3.910147e+06,2.491363e+06,67341.0,264668.0,656938.0,5154819.0,Saint-Alban
12,51.014440,2.135000,3.770284e+06,3.130005e+06,39517.0,252925.0,424200.0,5247974.0,Gravelines
9,47.903108,7.563036,4.138736e+06,2.757394e+06,50346.0,241690.0,962487.0,7442318.0,Fessenheim
2,45.798330,5.270830,3.953121e+06,2.532309e+06,42646.0,189795.0,705931.0,6137194.0,Bugey
7,44.633060,4.756670,3.904536e+06,2.405940e+06,52694.0,139678.0,261372.0,3195321.0,Cruas
18,44.335698,4.722490,3.899621e+06,2.373216e+06,57667.0,129654.0,307186.0,2695950.0,Tricastin
15,49.976110,1.211940,3.692281e+06,3.022563e+06,24240.0,103360.0,169571.0,2562167.0,Penly
4,47.230600,0.170280,3.578537e+06,2.729031e+06,29505.0,86334.0,171913.0,2719971.0,Chinon-B
11,44.106670,0.845280,3.588500e+06,2.377804e+06,18419.0,80524.0,189483.0,2358423.0,Golfech
